In [ ]:
import pickle
import numpy as np

In [ ]:
predicted_rocs = pickle.load(open('predicted_roc_dict.pickle', 'rb'))
print(predicted_rocs)

In [ ]:
predicted_rocs_list = [[k, v] for k, v in predicted_rocs.items()]
predicted_rocs_df = pd.DataFrame(predicted_rocs_list, columns = ['ISIN','CIT'])
predicted_rocs_df.sort_values('ISIN', axis=0, inplace=True)
predicted_rocs_df['CIT'] = predicted_rocs_df['CIT'].str.upper()
predicted_rocs_df.head()

In [ ]:
import pandas as pd

all_cities = pd.read_csv( '../train/outcome/ROC_fullList.csv', header=0, sep=";", quoting=1, thousands=",")
all_cities.head()

In [ ]:
print(len(all_cities['CIT_NM'].unique()))

In [ ]:
ref_cities = all_cities['CIT_NM'].as_matrix()
rocs = predicted_rocs_df.as_matrix()

In [ ]:
print(ref_cities[0])
print(rocs.shape)

In [2]:
import re
import pandas as pd
import numpy as np
import csv
import pickle

def eliminate_duplicates(cities):
    return '|'.join(set(cities.split('|')))

def convert_roc(roc_dict_path, ref_cities_path, predicted_currencies, link_roc, output_csv):
    
    result = {}
    
    predicted_rocs = pickle.load(open(roc_dict_path, 'rb'))
    predicted_rocs_list = [[k, v] for k, v in predicted_rocs.items()]
    predicted_rocs_df = pd.DataFrame(predicted_rocs_list, columns = ['ISIN','CIT'])
    predicted_rocs_df.sort_values('ISIN', axis=0, inplace=True)
    predicted_rocs_df['CIT'] = predicted_rocs_df['CIT'].str.upper()
    
    opCurrency=np.load(predicted_currencies).item()
    op_curr = [[k, v] for k, v in opCurrency.items()]

    opCurr_df = pd.DataFrame(op_curr, columns = ['ISIN','OpCurrency'])
    opCurr_df['OpCurrency'] = [w.decode("utf-8").strip().upper() for w in opCurr_df['OpCurrency'].values]
    opCurr_df.sort_values('OpCurrency', axis=0, inplace=True)
    
    link_roc_df = pd.read_csv(link_roc, header=0, sep=";", quoting=1, thousands=",")
    link_roc_df.sort_values('Currency Code', axis=0, inplace = True)

    roc_fromCurrency = pd.merge(left = link_roc_df, right = opCurr_df, left_on = 'Currency Code', right_on = 'OpCurrency')
    roc_fromCurrency = roc_fromCurrency[['ISIN','Default ROC']]
#     roc_fromCurrency.sort_values('ISIN', axis=0, inplace = True)

    all_cities = pd.read_csv(ref_cities_path, header=0, sep=";", quoting=1, thousands=",")
   
    ref_cities = all_cities['CIT_NM'].as_matrix()
    rocs = predicted_rocs_df.as_matrix()
    op_curr_rocs = roc_fromCurrency.as_matrix()

    for roc in rocs:
        isin = roc[0]
        cities = roc[1].decode("utf-8").strip()

        for ref_c in ref_cities:
            ref_c = ref_c.strip()

            matches = re.findall(ref_c, cities)
            if matches:
                if(len(matches) > 1):
                    print('Found multiple matches for %s - %s' % ref_c, cities)
                
                city = matches[0]
                if city in ['TARGET 1', 'TARGET 2']:
                    city = 'TARGET HOLIDAYS'
                    
                try:
                    existing_cities = result[isin]
                    result[isin] = existing_cities + '|' + city
                except KeyError:
                    result[isin] = matches[0]
    
    for roc in op_curr_rocs:
        isin = roc[0]
        cities = roc[1].upper().strip()
        
        for ref_c in ref_cities:
            ref_c = ref_c.strip()

            matches = re.findall(ref_c, cities)
            if matches:
                if(len(matches) > 1):
                    print('Found multiple matches for %s - %s' % ref_c, cities)
                try:
                    existing_cities = result[isin]
                    city = matches[0]
                    
                    if city not in existing_cities:
                        if city in ['TARGET 1', 'TARGET 2']:
                            city = 'TARGET HOLIDAYS'
                        
                        result[isin] = existing_cities + '|' + city
                        
                except KeyError:
                    result[isin] = matches[0]
    
    new_result = {k : eliminate_duplicates(v) for k,v in result.items() }
    
    result_df = pd.DataFrame.from_dict(new_result, orient='index')
    result_df.to_csv(output_csv, index=True, header=False, quoting=csv.QUOTE_ALL)
    
    print('Saved results to %s' % output_csv)
    
    return result_df

r = convert_roc('predicted_roc_dict.pickle', \
                '../train/outcome/ROC_fullList.csv', \
                'currencyExtraction/predicted_opCurr_dict.npy', \
                'ROC/currency_ROC_fullList.csv', \
                'roc.csv')
print(r)

Saved results to roc.csv
                                                      0
XS1254236638                         HONG KONG|NEW YORK
XS1378387531                                  HONG KONG
CH0281541968           MELBOURNE|SYDNEY|LONDON|NEW YORK
XS1179183436                                   NEW YORK
XS1014627498             TOKYO|BRASILIA|LONDON|NEW YORK
XS1368564990                                   NEW YORK
XS1330396513                               LONDON|TOKYO
XS1374689054                               LONDON|TOKYO
XS1306639664             TOKYO|BRASILIA|LONDON|NEW YORK
XS1378451998                                  HONG KONG
XS1396347053        AUCKLAND|WELLINGTON|LONDON|NEW YORK
XS0848707062                                   NEW YORK
CH0295931403                                   NEW YORK
XS1395264499                         HONG KONG|NEW YORK
NL0011218706                            TARGET HOLIDAYS
CH0279922980                            TARGET HOLIDAYS
XS1072479048           

In [ ]:

x = 'LONDON'
y = 'LONDON NEW YORK TARGET HOLIDAYS TORONTO'
re.findall(x, y)

In [ ]:
opCurrency = np.load('currencyExtraction/predicted_opCurr_dict.npy').item()

# op_curr = pickle.load(open(roc_dict_path, 'rb'))
op_curr = [[k, v] for k, v in opCurrency.items()]

op_curr_df = pd.DataFrame(op_curr, columns = ['ISIN','CIT'])
op_curr_df.sort_values('ISIN', axis=0, inplace=True)
op_curr_df['CIT'] = op_curr_df['CIT'].str.upper()

op_curr_df.head()


In [ ]:
opCurrency=np.load('currencyExtraction/predicted_opCurr_dict.npy').item()

op_curr = [[k, v] for k, v in opCurrency.items()]

opCurr_df = pd.DataFrame(op_curr, columns = ['ISIN','OpCurrency'])
opCurr_df['OpCurrency'] = [w.decode("utf-8").strip() for w in opCurr_df['OpCurrency'].values]

opCurr_df.head()

roc_fromCurrency = pd.merge(left=link_roc_opCurr,right=opCurr_df,left_on='Currency Code', right_on='OpCurrency')

roc_fromCurrency.head()

col=['ISIN','Default ROC']

roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.head()



In [ ]:
x = {'AA':'LONDON|SYDNEY|MELBOURNE|SYDNEY|TOKYO'}

result = {k : eliminate_duplicates(v) for k,v in x.items() }

print(result)

In [ ]:
print(t)

In [ ]:
 link_roc_df = pd.read_csv('ROC/currency_ROC_fullList.csv', header=0, sep=";", quoting=1, thousands=",")
    

In [ ]:
link_roc_df.head()